In [27]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import math

In [50]:
df = pd.read_csv('./Precos_Imoveis.csv')
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


## Limpeza dos dados
* Eliminando dados duplicados
* Lidando com valores ausentes

In [51]:
df.drop_duplicates()
df.shape
## Não há linhas duplicadas

(1460, 81)

In [52]:
isna = df.isnull().sum()
isna = isna[isna>0]
isna.sort_values(ascending = False)

PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
FireplaceQu      690
LotFrontage      259
GarageYrBlt       81
GarageType        81
GarageFinish      81
GarageQual        81
GarageCond        81
BsmtFinType2      38
BsmtExposure      38
BsmtFinType1      37
BsmtCond          37
BsmtQual          37
MasVnrArea         8
MasVnrType         8
Electrical         1
dtype: int64

In [53]:
df.PoolQC.value_counts(dropna=False)

NaN    1453
Gd        3
Fa        2
Ex        2
Name: PoolQC, dtype: int64

In [54]:
## Mostrando que a área da pscina é sempre 0 quando PoolQC é nulo
print(df[df.PoolQC.isnull()].PoolArea.max())
print(df[df.PoolQC.isnull()].PoolArea.mean())

0
0.0


In [55]:
df.MiscFeature.value_counts(dropna=False)

NaN     1406
Shed      49
Gar2       2
Othr       2
TenC       1
Name: MiscFeature, dtype: int64

In [56]:
df.Fence.value_counts(dropna=False)

NaN      1179
MnPrv     157
GdPrv      59
GdWo       54
MnWw       11
Name: Fence, dtype: int64

## Dropando colunas com alto número de NAs?

Optou-se por não eliminar as colunas com alto número de NA, pois, de acordo com a descrição dos dados, esses NAs representam categorias como "Sem piscina", "Sem cerca", "Sem característica diversa"...

In [57]:
# df = df.drop(labels=['PoolQC', 'Fence', 'MiscFeature'], axis=1)

## Preenchendo NAs com outros valores

In [58]:
def fill_garage_year(row):
    year_garage = row.GarageYrBlt
    year_built = row.YearBuilt
    if pd.isnull(year_garage) and pd.notnull(year_built):
        return year_built
    else:
        return year_garage

In [59]:
df['GarageYrBlt'] = df.apply(fill_garage_year, axis=1)

In [60]:
isna = df.isnull().sum()
isna = isna[isna>0]
isna.sort_values(ascending = False)

PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
FireplaceQu      690
LotFrontage      259
GarageType        81
GarageFinish      81
GarageQual        81
GarageCond        81
BsmtExposure      38
BsmtFinType2      38
BsmtFinType1      37
BsmtCond          37
BsmtQual          37
MasVnrArea         8
MasVnrType         8
Electrical         1
dtype: int64

In [61]:
## Verificando se os valores nulos de MasVnrArea estão relacionados com o MasVnrType ser nulo
df[df['MasVnrType'].isnull()]['MasVnrArea'].value_counts(dropna=False)

NaN    8
Name: MasVnrArea, dtype: int64

In [62]:
## neste caso, MasVnrArea recebe o valor 0
def fill_mas_area(row):
    mas_type = row.MasVnrType
    mas_area = row.MasVnrArea
    if pd.isnull(mas_type) and pd.isnull(mas_area):
        return 0
    else:
        return mas_area
df['MasVnrArea'] = df.apply(fill_mas_area, axis=1)

In [63]:
# lidando com LotFrontage preenchenco com valores médios
df['LotFrontage'] = df['LotFrontage'].fillna(df['LotFrontage'].median())

In [64]:
# Removendo coluna 'Id'
df = df.drop(labels=['Id'], axis=1)

In [65]:
df.shape

(1460, 80)

## Lidado com dados categóricos

Apesar do dado ser do tipo numérico, a coluna MSSubClass é um dado categórico e os números são códigos para as categorias

In [66]:
categorical = df.select_dtypes(include=['object']).columns
categorical = np.concatenate((categorical, ['MSSubClass']))
categorical

array(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour',
       'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood',
       'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish',
       'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence',
       'MiscFeature', 'SaleType', 'SaleCondition', 'MSSubClass'],
      dtype=object)

In [67]:
df = pd.get_dummies(df, dummy_na=True, columns=categorical)

In [68]:
df.shape

(1460, 347)

In [69]:
df.head()

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,MSSubClass_70.0,MSSubClass_75.0,MSSubClass_80.0,MSSubClass_85.0,MSSubClass_90.0,MSSubClass_120.0,MSSubClass_160.0,MSSubClass_180.0,MSSubClass_190.0,MSSubClass_nan
0,65.0,8450,7,5,2003,2003,196.0,706,0,150,...,0,0,0,0,0,0,0,0,0,0
1,80.0,9600,6,8,1976,1976,0.0,978,0,284,...,0,0,0,0,0,0,0,0,0,0
2,68.0,11250,7,5,2001,2002,162.0,486,0,434,...,0,0,0,0,0,0,0,0,0,0
3,60.0,9550,7,5,1915,1970,0.0,216,0,540,...,1,0,0,0,0,0,0,0,0,0
4,84.0,14260,8,5,2000,2000,350.0,655,0,490,...,0,0,0,0,0,0,0,0,0,0


In [70]:
## Garantindo que, após tranformar as variáveis
## categóricas em dummies, não há mais valores NA

isna = df.isnull().sum()
isna = isna[isna>0]
isna.sort_values(ascending = False)

Series([], dtype: int64)

### Salvando arquivo em binario com Pickle

In [71]:
from pickle import Pickler, Unpickler

In [72]:
f = open('./imoves_df', 'wb')
Pickler(f).dump(df)
f.close()